In [1]:
import torch
import numpy as np
from torchvision import transforms, datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib  inline
import random
from tqdm import tqdm
import os
import pandas as pd
from PIL import Image
import cv2 as cv
from google.colab.patches import cv2_imshow # for image display in google colab


from google.colab import drive
drive.mount('/content/drive')

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# setting seed
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
!pip install -U nltk
import nltk
nltk.download('wordnet')

!pip install torchinfo

Mounted at /content/drive
     |████████████████████████████████| 1.5MB 27.5MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Normalizing caption text and arranging train, val and test captions

In [2]:
image_folder_path = "/content/drive/MyDrive/VR final project/Flickr8K/Flicker8k_Images/"
image_text_folder_path = "/content/drive/MyDrive/VR final project/Flickr8K/Flickr8k_text/"

import re
import string
def normalize_text(text):
  text = text.lower()

  # Remove punctuations
  exclude = set(string.punctuation)
  text = "".join(ch for ch in text if ch not in exclude)

  # Remove articles
  regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
  text = re.sub(regex, " ", text)

  # Remove extra white space
  text = " ".join(text.split())
  return text


def sep_image_name(x):
  y = str(x)
  return y[:-2]

def create_label_set(filename_lst, file_to_token_dict):
  label_lst = []
  for file in filename_lst:
    lst = file_to_token_dict[file]
    transformed_lst = []
    for i, sentence in enumerate(lst):
      transformed_lst.append("<sos> " + normalize_text(sentence) + " <eos>")
    label_lst.append(transformed_lst)
  return label_lst 

def create_table(image_folder_path, image_text_folder_path, token_type='Lemmatised'):
  # image and raw token caption
  image_rawToken_caption = pd.read_csv(os.path.join(image_text_folder_path, 'Flickr8k.token.txt'), sep="\t", header = None, names = ["image", "caption"])
  # image and lemmatized token caption
  image_lemmatize_caption = pd.read_csv(os.path.join(image_text_folder_path, 'Flickr8k.lemma.token.txt'), sep="\t", header = None, names = ["image", "caption"])

  train_image_list = pd.read_csv(os.path.join(image_text_folder_path, 'Flickr_8k.trainImages.txt'), sep='\t', header=None, names=['image'])
  val_image_list = pd.read_csv(os.path.join(image_text_folder_path, 'Flickr_8k.valImages.txt'), sep='\t', header=None, names=['image'])
  test_image_list = pd.read_csv(os.path.join(image_text_folder_path, 'Flickr_8k.testImages.txt'), sep='\t', header=None, names=['image'])

  image_rawToken_caption['imageName'] = image_rawToken_caption['image'].apply(sep_image_name)
  image_with_rawTokens = image_rawToken_caption.groupby('imageName')['caption'].agg(list)
  image_filename_to_rawtaglst = image_with_rawTokens.to_dict()

  image_lemmatize_caption['imageName'] = image_lemmatize_caption['image'].apply(sep_image_name)
  image_with_lemmTokens = image_lemmatize_caption.groupby('imageName')['caption'].agg(list)
  image_filename_to_lemmtaglst = image_with_lemmTokens.to_dict()

  train_item_lst = train_image_list['image'].to_list()
  val_item_lst = val_image_list['image'].to_list()
  test_item_lst = test_image_list['image'].to_list()

  if (token_type == 'Lemmatised'):
    train_tags_lst = create_label_set(train_item_lst, image_filename_to_lemmtaglst)
    val_tags_lst = create_label_set(val_item_lst, image_filename_to_lemmtaglst)
    test_tags_lst = create_label_set(test_item_lst, image_filename_to_lemmtaglst)
  else :
    train_tags_lst = create_label_set(train_item_lst, image_filename_to_rawtaglst)
    val_tags_lst = create_label_set(val_item_lst, image_filename_to_rawtaglst)
    test_tags_lst = create_label_set(test_item_lst, image_filename_to_rawtaglst)

  return train_tags_lst, val_tags_lst, test_tags_lst


lst_labels = create_table(image_folder_path, image_text_folder_path, token_type='non_Lemmatised') # not lemmatised
#lst_labels[0] -> train labels
# lst_labels[1] -> val labels
# lst_labels[2] -> test labels

Loading the train, validation and test tensor that was preprocessed and saved in drive

In [3]:
!cp -r  /content/drive/MyDrive/VR" "final" "project/TrainLoader.pth /content/
!cp -r /content/drive/MyDrive/VR" "final" "project/TestLoader.pth /content/
!cp -r /content/drive/MyDrive/VR" "final" "project/ValLoader.pth  /content/

In [4]:
TrainLoader = torch.load('TrainLoader.pth') # here batch size is 1 need to change

In [5]:
ValLoader = torch.load('ValLoader.pth')

In [6]:
TestLoader = torch.load('TestLoader.pth')

Creating Vocabulary from the given captions

In [7]:
# preparing data for LSTM

import itertools

class Vocab():
  def __init__(self):
    self.word_to_index = {}
    self.index_to_word = {}
    self.n_words = 0
    self.word_set = {}

  def buildVocab(self, label_lst):
    label_lst = list(itertools.chain.from_iterable(label_lst[0])) # building vocab from train data
    big_str = " ".join(label_lst)
    big_str = big_str.lower() # lower case
    self.word_set = set(big_str.split(" "))
    self.word_set.add("<UNK>")
    self.word_set.add("<PAD>")
    self.word_to_index = {word:index for index, word in enumerate(self.word_set)}
    self.index_to_word = {index:word for index, word in enumerate(self.word_set)}
    self.n_words = len(self.word_set)

  def transformLabelToToken(self, label_lst):
    transformed_lst = []
    for lst in label_lst:
      new_lst = []
      for lst_of_sent in lst:
        new_lst_of_sent = []
        for sentence in lst_of_sent:
          normalize_text(sentence)
          transformed_sent = list(map(lambda x: self.word_to_index[x] if x in self.word_to_index else self.word_to_index["<UNK>"], sentence.lower().split(" ")))
          new_lst_of_sent.append(transformed_sent)
        new_lst.append(new_lst_of_sent)
      transformed_lst.append(new_lst)
    return transformed_lst  

  def padTokens(self, token_lst, MAX_LEN):
    for i in range(len(token_lst)):
      for j in range(len(token_lst[i])):
        for k in range(len(token_lst[i][j])):
          if (len(token_lst[i][j][k]) < MAX_LEN):
            for x in range(MAX_LEN - len(token_lst[i][j][k])):
              token_lst[i][j][k].append(self.word_to_index["<PAD>"])

    return token_lst 


In [8]:
#googleNet = models.googlenet(pretrained=True)

resNet = models.resnet18(pretrained=True)
#vggNet = models.vgg16(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


Model architecture code

In [13]:
class EncoderCNN(nn.Module):
  def __init__(self, pretrained_model):
    """
    Using pretrained CNN architecture - try using VGGnet as given in the paper: 
    Fine tuning on the upper layers also possible
    """
    super(EncoderCNN, self).__init__()

    model_layers = list(pretrained_model.children())[:-2] # for googleNet -3, vggNet, resNet is -2
    self.pretrained_model = nn.Sequential(*model_layers)

    # fine-tuning is also possible 
    for param in self.pretrained_model.parameters():
      param.requires_grad = False    

  def forward(self, img_input):
    output = self.pretrained_model(img_input)  # (batch_size, img_depth, img_height, img_width)
    output = output.permute(0, 2, 3, 1).contiguous()  #(batch_size, img_height, img_width, img_depth)
    return output


class Attention(nn.Module):
  def __init__(self, encoder_size, hidden_size, attention_size):
    """
    encoder_size : encoder output size i.e image depth
    hidden_size: size of decoder's hidden vector at a single timestep 
    attention_size: size of attention feed forward layer
    """
    super(Attention, self).__init__()
    self.encoder_size = encoder_size
    self.hidden_size = hidden_size
    self.attention_size = attention_size

    self.U = nn.Linear(encoder_size, attention_size)
    self.W = nn.Linear(hidden_size, attention_size)
    self.V = nn.Linear(attention_size, 1)
    self.softmax = nn.Softmax(dim=1)
    self.tanh = nn.Tanh()

  def forward(self, encoder_out, decoder_hidden):
    # decoder_hidden is the hidden output of decoder at time step t-1
    # encoder_out shape: (Batch_size, num_pixels, encoder_size) #encoder_size=img_depth
    
    encoder_att = self.U(encoder_out)                 # encoder_att shape: (batch_size, num_pixels, attention_size)
    decoder_att = self.W(decoder_hidden)              # decoder_att shape: (batch_size, attention_size) 
    f_att = self.V(self.tanh(encoder_att + decoder_att.unsqueeze(1)))  # f_att shape: (batch_size, num_pixels, 1) (elementwise addition after broadcasting)
    f_att = f_att.squeeze(2)                                           # f_att shape: (batch_size, num_pixels)

    alpha = self.softmax(f_att)                                        # alphas shape: (batch_size, num_pixels)
    context_vector = (encoder_out*alpha.unsqueeze(2)).sum(dim=1)       # context_vector shape: (batch_size, encoder_size) (elementwise multiplication after broadcasting (before summation))

    return context_vector, alpha

class DecoderRNN(nn.Module):
  def __init__(self, embed_size, vocab_size, attention_size, encoder_size, hidden_size, dropout_prob):
    """
    embed_size : embedding size (output of embedding layer)
    vocab_size: size of vocabulary
    attention_size: size of attention feed forward layer
    encoder_size: size of encoder output feature i.e image_depth 
    decoder_size: size of decoder hidden feature i.e hidden size of lstm
    """
    super(DecoderRNN, self).__init__()
    self.embed_size = embed_size
    self.vocab_size = vocab_size
    self.attention_size = attention_size
    self.encoder_size = encoder_size
    self.hidden_size = hidden_size
    
    self.attention_net = Attention(encoder_size, hidden_size, attention_size).to(device)
    self.embedding = nn.Embedding(vocab_size, embed_size)
    
    # lstm_cell. For multilayers, define multiple such 
    self.lstm_cell = nn.LSTMCell(embed_size + encoder_size, hidden_size, bias=True)

    self.dropout = nn.Dropout(p = dropout_prob)
    self.linear = nn.Linear(hidden_size, vocab_size)  # final layer to give logits on vocabulary

    self.init_h = nn.Linear(encoder_size, hidden_size) # initialization of initial hidden state vector
    self.init_c = nn.Linear(encoder_size, hidden_size) # initialization of initial cell state vector

    self.f_beta = nn.Linear(hidden_size, encoder_size)  # gating feed forward function as stated in the paper
    self.sigmoid = nn.Sigmoid() # for gating function 

  def init_hidden(self, encoder_out):
    mean_encoder_out = encoder_out.mean(dim=1) # (batch_size, num_pixels, encoder_size) -> (batch_size, encoder_size)
    init_hidden_state = self.init_h(mean_encoder_out)
    init_cell_state = self.init_c(mean_encoder_out)
    return init_hidden_state, init_cell_state

  def forward(self, encoder_out, captions, MAX_LEN):
    """
    Uses teacher forcing technique
    """
    # encoder_out shape: (batch_size, image_height, image_width, image_depth)
    batch_size = encoder_out.size(0)
    encoder_size = encoder_out.size(-1)
    encoder_out = encoder_out.view(batch_size, -1, encoder_size) # now the shape: (batch_size, num_pixels, encoder_size)
    
    
    embeddings = self.embedding(captions[:, :-1])      # embeddings shape: (batch_size, seq_len->MAX_LEN-1, embed_size)
    hidden_state, cell_state = self.init_hidden(encoder_out)

    out_logits = torch.zeros(batch_size, MAX_LEN-1, self.vocab_size).to(device)   # (batch_size, seq_len->MAX_LEN-1, vocab_size)
    alphas = torch.zeros(batch_size, MAX_LEN-1, encoder_out.size(1)).to(device)      # alphas are needed for regularization

    for t in range(MAX_LEN-1):
      context_vector, alpha = self.attention_net(encoder_out, hidden_state)
      beta = self.sigmoid(self.f_beta(hidden_state))
      context_vector = beta*context_vector
      hidden_state, cell_state = self.lstm_cell(torch.cat([embeddings[:, t, :], context_vector], dim=1), (hidden_state, cell_state))
      output= self.linear(self.dropout(hidden_state))
      out_logits[:, t, :] = output
      alphas[:, t, :] = alpha

    return out_logits, alphas


  def mix_technique(self, encoder_out, captions, MAX_LEN):
    """
    Uses teacher forcing technique for first 15 time steps and non teacher forcing afterwards
    """
    # encoder_out shape: (batch_size, image_height, image_width, image_depth)
    batch_size = encoder_out.size(0)
    encoder_size = encoder_out.size(-1)
    encoder_out = encoder_out.view(batch_size, -1, encoder_size) # now the shape: (batch_size, num_pixels, encoder_size)
    
    
    embeddings = self.embedding(captions[:, :-1])      # embeddings shape: (batch_size, seq_len->MAX_LEN-1, embed_size)
    hidden_state, cell_state = self.init_hidden(encoder_out)   # hidden_state shape: (batch_size, hidden_size)

    out_logits = torch.zeros(batch_size, MAX_LEN-1, self.vocab_size).to(device)   # (batch_size, seq_len->MAX_LEN-1, vocab_size)
    alphas = torch.zeros(batch_size, MAX_LEN-1, encoder_out.size(1)).to(device)      # alphas are needed for regularization

    for t in range(MAX_LEN-1):
      if (t<15): # 15
        input = embeddings[:, t, :]
      else:
        input = prev_embedding

      context_vector, alpha = self.attention_net(encoder_out, hidden_state)
      beta = self.sigmoid(self.f_beta(hidden_state))
      context_vector = beta*context_vector
      hidden_state, cell_state = self.lstm_cell(torch.cat([input, context_vector], dim=1), (hidden_state, cell_state))
      output= self.linear(self.dropout(hidden_state))
      out_logits[:, t, :] = output
      alphas[:, t, :] = alpha
      output = nn.LogSoftmax(dim=1)(output)
      output = output.argmax(1) 
      prev_embedding = self.embedding(output.unsqueeze(1))

    return out_logits, alphas    

  def generateCaptions(self, encoder_out, VocabObj, MAX_LEN):
    """
    Doesn't use teacher forcing technique -> used while decoding and testing
    Used for batch_size = 1
    """
    batch_size = encoder_out.size(0)
    encoder_size = encoder_out.size(-1)
    encoder_out = encoder_out.view(batch_size, -1, encoder_size)

    # embedding for '<sos>'
    sos_tensor = VocabObj.word_to_index["<sos>"]
    sos_tensor = torch.tensor([[sos_tensor] for l in range(batch_size)], dtype=int)  # sos_tensor shape: (batch_size, 1)
    sos_tensor = sos_tensor.to(device)
    embedding = self.embedding(sos_tensor) # initial input embedding

    hidden_state, cell_state = self.init_hidden(encoder_out)

    out_logits = torch.zeros(batch_size, MAX_LEN-1, self.vocab_size).to(device)
    
    generated_captions = ["<sos>"]

    with torch.no_grad():
      for t in range(MAX_LEN-1):
        context_vector, alpha = self.attention_net(encoder_out, hidden_state)
        beta = self.sigmoid(self.f_beta(hidden_state))
        context_vector = beta*context_vector
        hidden_state, cell_state = self.lstm_cell(torch.cat([embedding.squeeze(1), context_vector], dim=1), (hidden_state, cell_state))
        output= self.linear(hidden_state)  # output shape: (batch_size, vocab_size)
        out_logits[:, t, :] = output
        output = nn.LogSoftmax(dim=1)(output)
        output = output.argmax(1)  # output shape: (batch_size)
        embedding = self.embedding(output.unsqueeze(1))  # no teacher forcing

        generated_captions.append(VocabObj.index_to_word[output.item()])
        if (VocabObj.index_to_word[output.item()] == "<eos>"):
          break

    #print(f"output: {output}") 
    return generated_captions


class ImageCaptionModel(nn.Module):
  def __init__(self, pretrained_model, embed_size, vocab_size, attention_size, encoder_size, hidden_size, dropout_prob):
    super(ImageCaptionModel, self).__init__()
    self.encoder = EncoderCNN(pretrained_model)
    self.decoder = DecoderRNN(embed_size, vocab_size, attention_size, encoder_size, hidden_size, dropout_prob)
  
  def forward(self, img_input, captions, MAX_LEN):
    return self.decoder(self.encoder(img_input), captions, MAX_LEN)

  def mix_technique(self, img_input, captions, MAX_LEN):
    return self.decoder(self.encoder(img_input), captions, MAX_LEN)

  def generateCaptions(self, img_input, VocabObj, MAX_LEN):
    return self.decoder.generateCaptions(self.encoder(img_input), VocabObj, MAX_LEN)




Visualizing model summary and memory details

In [ ]:
# EMBED_SIZE = 256  
# ENCODER_SIZE = 512
# HIDDEN_SIZE = 512
# ATTENTION_SIZE = 256
# VOCAB_SIZE = 7631
# LSTM_DROPOUT_PROB = 0.6
# NUM_EPOCHS = 20
# LEARNING_RATE = 1e-4

# model = ImageCaptionModel(resNet, EMBED_SIZE, VOCAB_SIZE, ATTENTION_SIZE, ENCODER_SIZE, HIDDEN_SIZE, LSTM_DROPOUT_PROB)


# from torchinfo import summary


# summary(model, [(32, 3, 256, 256), (32, 163)], dtypes=[torch.float, torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
ImageCaptionModel                        --                        --
├─EncoderCNN: 1-1                        [32, 8, 8, 512]           --
│    └─Sequential: 2-1                   [32, 512, 8, 8]           --
│    │    └─Conv2d: 3-1                  [32, 64, 128, 128]        (9,408)
│    │    └─BatchNorm2d: 3-2             [32, 64, 128, 128]        (128)
│    │    └─ReLU: 3-3                    [32, 64, 128, 128]        --
│    │    └─MaxPool2d: 3-4               [32, 64, 64, 64]          --
│    │    └─Sequential: 3-5              [32, 64, 64, 64]          (147,968)
│    │    └─Sequential: 3-6              [32, 128, 32, 32]         (525,568)
│    │    └─Sequential: 3-7              [32, 256, 16, 16]         (2,099,712)
│    │    └─Sequential: 3-8              [32, 512, 8, 8]           (8,393,728)
├─DecoderRNN: 1-2                        [32, 162, 7631]           --
│    └─Embedding: 2-2                    [32,

Training function

In [14]:
def train(model, TrainLoader, TrainCaptionLoader, ValLoader, ValCaptionLoader, num_epochs, learning_rate, VocabObj, MAX_LEN, regularizer_const=1, teacher_forcing=True):
  model = model.to(device)
  criterion = nn.CrossEntropyLoss(ignore_index = VocabObj.word_to_index["<PAD>"])
  optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, momentum=0.05)

  train_loss_lst = []
  val_loss_lst = []

  for epoch in range(num_epochs):
    model.train()
    caption_loader = iter(TrainCaptionLoader)
    train_loss = 0
    for idx, img in tqdm(enumerate(TrainLoader), total=len(TrainLoader), position=0, leave=True):
      img = img.to(device)
      caption = next(caption_loader)
      caption = caption.to(device)

      batch_size = img.size(0)

      if (teacher_forcing):
        output_logits, alphas = model(img, caption, MAX_LEN)
      else:
        output_logits, alphas = model.mix_technique(img, caption, MAX_LEN)

      optimizer.zero_grad()
      loss = criterion(output_logits.permute(0, 2, 1).contiguous(), caption[:, 1:])
      loss += regularizer_const*((1. - alphas.sum(dim=1))**2).mean() # regularization as per the paper 
      loss.backward()
      optimizer.step()

    train_loss_lst.append(loss/batch_size)

    # validation loop
    if (epoch == num_epochs-1):
      model.eval()
      with torch.no_grad():
        val_caption_loader = iter(ValCaptionLoader)
        val_loss = float('inf')
        for idx, val_img in tqdm(enumerate(ValLoader), total=len(ValLoader), position=0, leave=True):
          val_img = val_img.to(device)
          batch_size = val_img.size(0)
          val_caption = next(val_caption_loader)
          for y in range(5):  
            val_cap = val_caption[y].to(device)
            val_cap = val_cap.unsqueeze(0)

            if (teacher_forcing):
              val_out_logits, alphas = model(val_img, val_cap, MAX_LEN)
            else:
              val_out_logits, alphas = model.mix_technique(val_img, val_cap, MAX_LEN)

            tmp_val_loss = criterion(val_out_logits.permute(0, 2, 1).contiguous(), val_cap[:, 1:])
            tmp_val_loss += regularizer_const*((1. - alphas.sum(dim=1))**2).mean()

            val_loss = min(val_loss, tmp_val_loss)

        val_loss_lst.append(val_loss/batch_size) 
    else:
      val_loss_lst.append(torch.zeros(1))

    print(f"Epoch: {epoch+1}/{num_epochs}, train loss: {train_loss_lst[epoch].item(): .4f}, validation loss: {val_loss_lst[epoch].item(): .4f}")

  return train_loss_lst, val_loss_lst


def showPlot(train_loss_lst, val_loss_lst):
  plt.figure(figsize=(8, 8))
  plt.plot(train_loss_lst)
  plt.plot(val_loss_lst)
  plt.title('loss per epoch')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend(['Training loss', 'Validation loss'])
  plt.savefig('Plot_of_loss_per_epoch.jpg')

Evaluation functions

In [23]:
# outputing captions -> Greedy method
def outputCaptions(model, Loader, VocabObj, MAX_LEN):
  # Loader can be ValLoader or TestLoader (batch_size is assumed to be 1 in this case)
  model.eval()
  output_captions = []
  with torch.no_grad():
    for idx, img in enumerate(Loader):
      img = img.to(device)
      caption_out = model.generateCaptions(img, VocabObj, MAX_LEN)
      output_captions.append(" ".join(caption_out))
  return output_captions


# BLEU eval metric
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate.meteor_score import meteor_score
def BLEU(reference_doc, sentence_list):
  # reference_doc 3-dm list
  # sentence_list is list of lists of sentence tokens 2-dm list
  bleuScore1 = corpus_bleu(reference_doc, sentence_list, weights=(1, 0, 0, 0))
  bleuScore2 = corpus_bleu(reference_doc, sentence_list, weights=(0.5, 0.5, 0, 0))
  bleuScore3 = corpus_bleu(reference_doc, sentence_list, weights=(1/3, 1/3, 1/3, 0))
  bleuScore4 = corpus_bleu(reference_doc, sentence_list, weights=(0.25, 0.25, 0.25, 0.25))
  return bleuScore1, bleuScore2, bleuScore3, bleuScore4


# Not a correct implementation of METEOR score; True one is more complex
def METEOR(reference_doc, sentence_list):
  score = 0
  for idx, _ in enumerate(sentence_list):
    score += meteor_score(reference_doc[idx], sentence_list[idx])
  return score/len(sentence_list) # averaging METEOR score

def Eval_score(label_lst, predicted_list):
  # label_lst -> list of lists of 5 sentences esp label_lst[2] #test
  sentence_list = [sent.split(" ") for sent in predicted_list]
  reference_doc = [[k.split(" ") for k in l] for l in label_lst] 
  bleuScore1, bleuScore2, bleuScore3, bleuScore4 = BLEU(reference_doc, sentence_list)
  meteorScore = METEOR(label_lst, predicted_list)

  
  print(f"BLEU-1 score: {bleuScore1: .5f}, BLEU-2 score: {bleuScore2: .5f}, BLEU-3 score: {bleuScore3: .5f}, BLEU-4 score: {bleuScore4: .5f}\nAverage METEOR score: {meteorScore: .5f}\n")


# evaluation
def evalResults(validation_captions, test_captions, label_lst):
  val_ref = label_lst[1]
  test_ref = label_lst[2]

  print(f"Validation metrics:\n")
  Eval_score(val_ref, validation_captions)

  print(f"\n Test metrics:\n")
  Eval_score(test_ref, test_captions)

Script

In [16]:
# script

BATCH_SIZE = 32

MAX_LEN = 163 
VocabObj = Vocab()
VocabObj.buildVocab(lst_labels)
token_lst = VocabObj.transformLabelToToken(lst_labels)
token_lst = VocabObj.padTokens(token_lst, MAX_LEN)

def prepareImageLoader(Loader, batch_size):
  lst = []
  for idx, img in enumerate(Loader):
    lst.append(img.squeeze(0)) 

  return DataLoader(dataset=lst, batch_size = batch_size, pin_memory=True)

In [17]:
TrainLoader = prepareImageLoader(TrainLoader, BATCH_SIZE)


In [18]:


def take_first_caption(token_lst):
  lst_tokens = []
  for lst in token_lst:
    lst_tokens.append(torch.tensor(lst[0]))
  return lst_tokens

def take_all_captions(token_lst):
  lst_tokens = []
  for lst in token_lst:
    for x in lst:
      lst_tokens.append(torch.tensor(x))
  return lst_tokens

TrainCaptionLoader = DataLoader(dataset = take_first_caption(token_lst[0]), batch_size = BATCH_SIZE)
ValCaptionLoader = DataLoader(dataset = take_all_captions(token_lst[1]), batch_size = 5)



EMBED_SIZE = 256 
ENCODER_SIZE = 512 
HIDDEN_SIZE = 512  
ATTENTION_SIZE = 256
VOCAB_SIZE = VocabObj.n_words
LSTM_DROPOUT_PROB = 0.6 
NUM_EPOCHS = 20 
LEARNING_RATE = 1e-4  

model = ImageCaptionModel(resNet, EMBED_SIZE, VOCAB_SIZE, ATTENTION_SIZE, ENCODER_SIZE, HIDDEN_SIZE, LSTM_DROPOUT_PROB)
model = model.to(device)


In [19]:
VocabObj.n_words

7631

In [20]:
train_loss_lst, val_loss_lst = train(model, TrainLoader, TrainCaptionLoader, ValLoader, ValCaptionLoader, NUM_EPOCHS, LEARNING_RATE, VocabObj, MAX_LEN, regularizer_const=1.4, teacher_forcing=True)

  0%|          | 0/188 [00:00<?, ?it/s]

Epoch: 1/20, train loss:  0.5666, validation loss:  0.0000


  0%|          | 0/188 [00:00<?, ?it/s]

Epoch: 2/20, train loss:  0.5436, validation loss:  0.0000


  0%|          | 0/188 [00:00<?, ?it/s]

...

Epoch: 18/20, train loss:  0.4150, validation loss:  0.0000


  0%|          | 0/188 [00:00<?, ?it/s]

Epoch: 19/20, train loss:  0.4123, validation loss:  0.0000


100%|██████████| 1000/1000 [07:36<00:00,  2.19it/s]

Epoch: 20/20, train loss:  0.4028, validation loss:  3.9383


In [21]:
#showPlot(train_loss_lst, val_loss_lst)

In [22]:
validation_captions = outputCaptions(model, ValLoader, VocabObj, MAX_LEN)
test_captions = outputCaptions(model, TestLoader, VocabObj, MAX_LEN)

In [29]:
evalResults(validation_captions, test_captions, lst_labels)

Validation metrics:

BLEU-1 score:  0.58859, BLEU-2 score:  0.35420, BLEU-3 score:  0.22794, BLEU-4 score:  0.14519
Average METEOR score:  0.38821


 Test metrics:

BLEU-1 score:  0.57993, BLEU-2 score:  0.34465, BLEU-3 score:  0.21268, BLEU-4 score:  0.12834
Average METEOR score:  0.38609



In [30]:
validation_captions

['<sos> boy in red shirt is sitting on skateboard <eos>',
 '<sos> man is climbing rock rock <eos>',
 '<sos> two dogs are playing in field <eos>',
 '<sos> black and white dog is jumping over water <eos>',
 '<sos> man in blue shirt is standing on rock <eos>',
 '<sos> baby is sitting in pool <eos>',
 '<sos> group of people are standing in front of people <eos>',
 '<sos> boy in blue shirt is running on grass <eos>',
 '<sos> boy in red shirt is sitting on skateboard <eos>',
 '<sos> boy in red shirt is standing on his head <eos>',
 '<sos> boy in blue shirt is running through field <eos>',
 '<sos> boy in blue shirt is playing with ball <eos>',
 '<sos> black dog is jumping in water <eos>',
 '<sos> boy in blue shirt is standing on rock <eos>',
 '<sos> man is riding on skateboard trick <eos>',
 '<sos> boy is jumping over water <eos>',
 '<sos> man in blue shirt is standing on his bike <eos>',
 '<sos> black dog is running through water <eos>',
 '<sos> man in red shirt is standing on rock <eos>',
 

Loading Subjective image tensor data from the drive


In [31]:
subject_img_folder = "/content/drive/MyDrive/VR final project/subjective_img_loader.pth"

subjective_img_loader = torch.load(subject_img_folder)

Testing the model on subjective images

In [32]:
subjective_captions = outputCaptions(model, subjective_img_loader, VocabObj, MAX_LEN)

In [33]:
subjective_captions

['<sos> boy is sitting on skateboard on skateboard <eos>',
 '<sos> group of people are riding on street <eos>',
 '<sos> group of people are standing in front of water <eos>',
 '<sos> group of people are standing on beach <eos>',
 '<sos> brown dog is jumping in air <eos>']